# 🚀 Enhance TinyLlama with Advanced Fine-Tuning on Diverse Data

In [ ]:
!pip install -q bitsandbytes accelerate datasets loralib peft transformers trl


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False


In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:
import pandas as pd
from datasets import Dataset

# Step 1: Read from output.csv
df = pd.read_csv("Mental_Health_FAQ.csv")  # Make sure 'question' and 'answer' columns are present

# Step 2: Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Step 3: Define formatting function
def format_instruction(example):
    return f"### Instruction:\n{example['Instruction']}\n\n### Response:\n{example['Response']}"

# Step 4: Apply formatting
dataset = dataset.map(lambda x: {"text": format_instruction(x)})

# Optional: Preview
print(dataset[0])


In [ ]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [ ]:
training_args = TrainingArguments(
    output_dir="./tinyllama-qlora-support-bot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_dir="./logs",
    num_train_epochs=15,
    logging_steps=10,
    save_total_limit=2,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_8bit"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


In [ ]:
model.save_pretrained("tinyllama-qlora-support-bot")
tokenizer.save_pretrained("tinyllama-qlora-support-bot")


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

instruction = "What does it mean to have a mental illness?"
prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"

output = pipe(prompt, max_new_tokens=100)
print(output[0]['generated_text'])


In [ ]:
import shutil
shutil.make_archive("tinyllama-qlora-support-bot", 'zip', "tinyllama-qlora-support-bot")


In [ ]:
from google.colab import files

# Download the zipped model to your local system
files.download("tinyllama-qlora-support-bot.zip")

In [ ]:
pip install -q gradio

In [ ]:
# Mount your Google Drive (optional)
# from google.colab import drive
# drive.mount('/content/drive')

# Unzip your fine-tuned model
!unzip -o tinyllama-qlora-support-bot.zip -d fine_tuned_model


## 🎨 Interactive Web Interface

Creating a user-friendly Gradio interface for interacting with our mental health assistant.

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# Load your fine-tuned Mental Illness Assistant model
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

def format_prompt(instruction):
    return f"### Instruction:\n{instruction}\n\n### Response:\n"

def chat_response(message, chat_history):
    prompt = format_prompt(message)
    output = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)
    generated_text = output[0]['generated_text']
    response = generated_text.split("### Response:\n")[-1].strip()
    chat_history.append((message, response))
    return "", chat_history

# Gradio UI for Mental Illness Assistant
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧠 Mental Health Assistant\nAsk questions about mental illness, symptoms, and care.")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Question", placeholder="e.g., What does it mean to have anxiety?", lines=1)
    submit = gr.Button("Submit")
    clear = gr.Button("Clear Chat")

    submit.click(chat_response, [msg, chatbot], [msg, chatbot])
    msg.submit(chat_response, [msg, chatbot], [msg, chatbot])  # Also allow Enter key
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


In [ ]:
# Upload CSV to Google Colab (from local machine)
from google.colab import files
import pandas as pd

# Method 1: Upload directly from your computer
uploaded = files.upload()

# Get the filename (assuming you upload just one file)
filename = next(iter(uploaded))

In [ ]:
import pandas as pd
df=pd.read_csv("Mental_Health_FAQ.csv")
print(df.columns)  # Should show: Index(['Questions', 'Answers'], dtype='object')
